In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Parameters
num_nodes = 1000
num_edges = 5000
delta = 0.02
topologies = ['RN', 'BAN']
l0_values = range(1, 7)
num_samples = 1000
max_attempts_per_l0 = 10000  # Max attempts to find pairs for each l0

def generate_topology(topology):
    np.random.seed(42)
    if topology == 'RN':
        p = 2 * num_edges / (num_nodes * (num_nodes - 1))
        G = nx.erdos_renyi_graph(num_nodes, p)
    elif topology == 'BAN':
        m = num_edges // num_nodes
        G = nx.barabasi_albert_graph(num_nodes, m)
    else:
        raise ValueError("Unsupported topology")
    # Ensure the graph has exactly num_edges
    while G.number_of_edges() < num_edges:
        u, v = np.random.choice(num_nodes, 2, replace=False)
        if not G.has_edge(u, v):
            G.add_edge(u, v)
    return G

def find_paths_method1(G, s, d, max_k=3):
    try:
        paths = nx.shortest_simple_paths(G, s, d)
        path_lengths = []
        for i, path in enumerate(paths):
            if i >= max_k:
                break
            path_lengths.append(len(path) - 1)
        return path_lengths
    except nx.NetworkXNoPath:
        return []

def calculate_C(l0_p, l1_p, l2_p, delta):
    numerator = (1 - (1/6)*(3*l0_p + 3*l1_p + 4*l2_p)*delta +
                (1/18)*(4*l0_p*l1_p + 5*l1_p*l2_p + 5*l2_p*l0_p)*delta**2 -
                (7/54)*l0_p*l1_p*l2_p*delta**3)
    denominator = (1 - (1/6)*(3*l0_p + 3*l1_p + 2*l2_p)*delta +
                  (1/18)*(6*l0_p*l1_p + 4*l1_p*l2_p + 4*l2_p*l0_p)*delta**2 -
                  (4/27)*l0_p*l1_p*l2_p*delta**3)
    return numerator / denominator if denominator != 0 else 0.0

def method1_simulation(G, topology):
    results = {l0: [] for l0 in l0_values}
    for l0 in l0_values:
        print(f"Method 1 ({topology}): Processing l0={l0}")
        pairs = set()
        attempts = 0
        with tqdm(total=num_samples, desc=f'l0={l0}', leave=False) as pbar:
            while len(results[l0]) < num_samples and attempts < max_attempts_per_l0:
                attempts += 1
                s = np.random.choice(G.nodes())
                d = np.random.choice(G.nodes())
                if s == d or (s, d) in pairs or (d, s) in pairs:
                    continue
                try:
                    spl = nx.shortest_path_length(G, s, d)
                except nx.NetworkXNoPath:
                    continue
                if spl != l0:
                    continue
                pairs.add((s, d))
                pairs.add((d, s))
                path_lengths = find_paths_method1(G, s, d, max_k=3)
                if len(path_lengths) < 3:
                    continue
                sorted_lengths = sorted(path_lengths, reverse=True)[:3]
                l0_p, l1_p, l2_p = sorted_lengths[0], sorted_lengths[1], sorted_lengths[2]
                C = calculate_C(l2_p, l1_p, l0_p, delta)
                results[l0].append(C)
                pbar.update(1)
        results[l0] = np.mean(results[l0]) if results[l0] else 0.0
    return results

def purification_deutsch(C1, C2):
    return (13 + 14*C1 + 14*C2 + 40*C1*C2) / (41 + 4*C1 + 4*C2 + 32*C1*C2)

def get_alternate_paths_method2(G, u, v, main_path):
    forbidden = set(main_path) - {u, v}
    subgraph = G.copy()
    subgraph.remove_nodes_from(forbidden)
    paths = []
    try:
        for path in nx.all_simple_paths(subgraph, u, v, cutoff=10):
            if len(path) >= 2:
                paths.append(len(path) - 1)
    except nx.NetworkXNoPath:
        pass
    if G.has_edge(u, v):
        paths.append(1)
    return paths

def method2_simulation(G):
    results = {l0: [] for l0 in l0_values}
    for l0 in l0_values:
        print(f"Method 2: Processing l0={l0}")
        pairs = set()
        attempts = 0
        with tqdm(total=num_samples, desc=f'l0={l0}', leave=False) as pbar:
            while len(results[l0]) < num_samples and attempts < max_attempts_per_l0:
                attempts += 1
                s = np.random.choice(G.nodes())
                d = np.random.choice(G.nodes())
                if s == d or (s, d) in pairs or (d, s) in pairs:
                    continue
                try:
                    path = nx.shortest_path(G, s, d)
                except nx.NetworkXNoPath:
                    continue
                if len(path) - 1 != l0:
                    continue
                pairs.add((s, d))
                pairs.add((d, s))
                link_Cs = []
                valid = True
                for i in range(len(path) - 1):
                    u_node = path[i]
                    v_node = path[i+1]
                    alt_paths = get_alternate_paths_method2(G, u_node, v_node, path)
                    if not alt_paths:
                        valid = False
                        break
                    sorted_lengths = sorted(alt_paths, reverse=True)
                    path_Cs = []
                    for plen in sorted_lengths:
                        c_es = np.random.uniform(0.97, 0.99, plen)
                        c_path = np.prod(c_es)
                        path_Cs.append(c_path)
                    if not path_Cs:
                        valid = False
                        break
                    current_C = path_Cs[0]
                    for c in path_Cs[1:]:
                        current_C = purification_deutsch(current_C, c)
                    link_Cs.append(current_C)
                if not valid or not link_Cs:
                    continue
                final_C = link_Cs[0]
                for c in link_Cs[1:]:
                    final_C = (final_C + c + 2 * final_C * c - 1) / 3
                    final_C = max(0.0, final_C)
                results[l0].append(final_C)
                pbar.update(1)
        results[l0] = np.mean(results[l0]) if results[l0] else 0.0
    return results

def main():
    plt.figure(figsize=(12, 8))
    markers = ['o', 's', '^', 'D', 'v']
    colors = ['blue', 'orange']
    for idx, topology in enumerate(topologies):
        print(f"\n=== Processing {topology} Topology ===")
        G = generate_topology(topology)
        print(f"Running Method 1 (MEP) for {topology}...")
        m1_results = method1_simulation(G, topology)
        print(f"Running Method 2 (Link-Level MEP) for {topology}...")
        m2_results = method2_simulation(G)
        
        x = list(l0_values)
        y_m1 = [m1_results[l0] for l0 in l0_values]
        y_m2 = [m2_results[l0] for l0 in l0_values]
        
        plt.plot(x, y_m1, marker=markers[idx*2], color=colors[idx], linestyle='-', label=f'MEP ({topology})')
        plt.plot(x, y_m2, marker=markers[idx*2+1], color=colors[idx], linestyle='--', label=f'Link-Level MEP ({topology})')
    
    plt.xlabel('Shortest Path Length (l₀)')
    plt.ylabel('Average Concurrence')
    plt.title('Comparison of MEP and Link-Level MEP Strategies')
    plt.legend()
    plt.grid(True)
    plt.ylim(0.85, 1.0)
    plt.show()

if __name__ == "__main__":
    main()